# Boilerplate

There's probably a better way than what follows

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')

import asyncio
import random # for nonce generation

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

What can we run?

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__')]

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'new_transport_interface',
 'nwv2_python_wrapper']

RequestAction packet, Connect

In [3]:
ra_connect = RequestActionW("connect", name="Siona", client_version="0.3.2")
ra_connect

Connect { name: "Siona", client_version: "0.3.2" }

Packet creation with no cookie or response ack

In [4]:
sequence = 0
response_ack = None
cookie = None
pkt_ra_connect = PacketW("request", sequence=sequence, response_ack=response_ack, action=ra_connect, cookie=cookie)

assert pkt_ra_connect.request("sequence") == sequence
assert pkt_ra_connect.request("response_ack") == response_ack
assert pkt_ra_connect.request("cookie") == cookie
assert pkt_ra_connect.request("action") == ra_connect

Connect { name: "Siona", client_version: "0.3.2" }
Connect { name: "Siona", client_version: "0.3.2" }


(True, 'fail')

In [6]:
endpoint = EndpointW("157.230.134.224:2016")
endpoint

Endpoint(157.230.134.224:2016)

GetStatus packet

In [7]:
random_nonce = random.getrandbits(64)
ping = PacketW("getstatus", ping_nonce=random_nonce)
ping

GetStatus { ping: PingPong { nonce: 8153870264903614398 } }

In [8]:
ping_nonce = ping.get_status("ping_nonce")
ping_nonce
assert ping_nonce == random_nonce

Status packet

In [9]:
pong_nonce = ping_nonce
server_version = "7.8.9"
player_count = 20
room_count = 10
server_name = "nwv2_python_wrapper_server"
pong = PacketW("status", pong_nonce=pong_nonce, server_version=server_version, player_count=player_count, room_count=room_count, server_name=server_name)
pong

Status { pong: PingPong { nonce: 8153870264903614398 }, server_version: "7.8.9", player_count: 20, room_count: 10, server_name: "nwv2_python_wrapper_server" }

Verify Status and GetStatus agree with each other

In [10]:
assert pong.status() == pong_nonce
assert pong.status("pong_nonce") == pong_nonce
assert pong.status("server_version") == server_version
assert pong.status("player_count") == player_count
assert pong.status("room_count") == room_count
assert pong.status("server_name") == server_name
try:
    pong.status("does_not_exist")
except ValueError:
    assert True
except Exception:
    assert False